# WEB SCRAPING 

### Imports and Libraries

In [ ]:
#working with images
! pip install opencv-python
from matplotlib import image
from matplotlib import pyplot
from skimage import io
import cv2
import os

#--Web scraping packages
from bs4 import BeautifulSoup
import requests
import itertools
from urllib.request import Request, urlopen
import urllib.request

#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np
from urllib.request import urlopen

#barrita de progreso
from tqdm.notebook import tqdm
from time import sleep 

#Visualize data
%pylab inline
plt.style.use('seaborn-talk')

### Functions we will use

In [ ]:
#Functions that gets a dictionary (setas_dct) from a webpage with the url "https://www.fungipedia.org....""
#with all mushrooms with their name and url.

def get_mushrooms_info():
    mushroom_dct = {}

    for idx in tqdm(range(0, 541, 20)):
        url = f"https://www.fungipedia.org/hongos.html?start={idx}"
        req = Request(
            url,
            headers={"User-Agent": "Safari/537.36"},
        )

        html = urlopen(req)
        soup = BeautifulSoup(html, "html.parser")

        mushrooms = soup.find_all(class_="gris")
        for m in mushrooms:
            mushroom_dct[m.get("title")] = m.get("href")

    return mushroom_dct

#setas_dct is a dictionary with Info about all mushrooms in the webpage (542)
#setas_dct_short is a dictionary with Info about the first 10 mushrooms in the webpage (10)

setas_dct = get_mushrooms_info()
print(f"Numero de setas encontradas: {len(setas_dct)}")

N = 10
print(f"Info de las primeras {N} setas")

setas_dct_short = dict(itertools.islice(setas_dct.items(), 10))
for name, link in setas_dct_short.items():
    print(f"Seta {name} en {link}")
    
#separate between names an urls mushrooms    
setas_nombres,setas_urls=list(setas_dct.keys()), list(setas_dct.values())

setas_nombres_10,setas_urls_10 = list(setas_dct_short.keys()), list(setas_dct_short.values())

In [ ]:
#function to download the images from an url and save in a folder.

def downloader(url,folder):
    
    os.chdir(os.path.join(os.getcwd(),'/home/dsc/FP_Mushrooms/Mushrooms_Classification/'))

    try:
        os.mkdir(os.path.join(os.getcwd(),folder))
    except:
        print('already created')

    os.chdir(os.path.join(os.getcwd(),'/home/dsc/FP_Mushrooms/Mushrooms_Classification/'+folder))
    
    response = requests.get(url)

    soup = BeautifulSoup(response.content,'html.parser')

    images = soup.find_all("img")
    
    number = 0

    for image in images[1::]:
    
        image_src = image['src']
    
        urllib.request.urlretrieve(image_src, str(number) + ".jpg")
    
        number += 1
        
for seta in tqdm(setas_nombres_10):
    
    url = 'https://www.google.com/search?q='+seta+'&tbm=isch&hl=es&sa=X&ved=2ahUKEwjw_M7z2NfuAhUMpRoKHbPQAokQgowBegQIARAX&biw=1905&bih=852'
    
    downloader(url,seta)
    
#volvemos a nuestro directorio    
os.chdir(os.path.join(os.getcwd(),'/home/dsc/FP_Mushrooms/Mushrooms_Classification/'))

### Ahora nos vamos a google colab